In [ ]:
# Import necessary modules
import pandas as pd
from src.data_processing import load_data, preprocess_data
from src.model_training import split_data, train_model, evaluate_model
from src.utils import save_model, load_model

# Load datasets
train_df, test_df = load_data('train.csv', 'test.csv')

# Preprocess data
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

# Split data
X_train, X_val, y_train, y_val = split_data(train_df)

# Train model
model = train_model(X_train, y_train)

# Evaluate model
accuracy = evaluate_model(model, X_val, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")

# Save model
save_model(model, 'titanic_rf_model.pkl')

# Load model for future use
loaded_model = load_model('titanic_rf_model.pkl')

# Predict on test data (for Kaggle submission)
test_features = test_df.drop(['PassengerId'], axis=1)
test_predictions = loaded_model.predict(test_features)

In [4]:
import sys
print(sys.path)


['d:\\sgc0326\\envs\\titanic_environment\\python311.zip', 'd:\\sgc0326\\envs\\titanic_environment\\DLLs', 'd:\\sgc0326\\envs\\titanic_environment\\Lib', 'd:\\sgc0326\\envs\\titanic_environment', '', 'd:\\sgc0326\\envs\\titanic_environment\\Lib\\site-packages', 'd:\\sgc0326\\envs\\titanic_environment\\Lib\\site-packages\\win32', 'd:\\sgc0326\\envs\\titanic_environment\\Lib\\site-packages\\win32\\lib', 'd:\\sgc0326\\envs\\titanic_environment\\Lib\\site-packages\\Pythonwin', 'd:\\sgc0326\\envs\\titanic_environment\\Lib\\site-packages\\setuptools\\_vendor']
